In [1]:
path=  'C:/Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/'
#import sys
#sys.path.append(path) 
path_=  'C:/Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/Base_Models'
import sys
sys.path.append(path_) 


# Regresor

In [2]:
from sklearn.linear_model import ElasticNet
from base_regressor import BaseRegressor
from Plotter import Plotter
from skopt.space import Real, Categorical, Integer

class ElasticNetRegressor(BaseRegressor):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="ElasticNet"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = ElasticNet
        if params is None:
            self.params = {
                'alpha': 0.2,
                'l1_ratio': 0.5,  # Proporción de L1 en la regularización
                'max_iter': 10000,
                # 'tol': 0.001
            }
        if params_space is None:
            self.params_space = {
                'alpha': Real(0.001, 0.5, prior='log-uniform'),
                'l1_ratio': Real(0.0, 1.0),  # Rango de 0 a 1 para la proporción de L1
                'max_iter': Integer(100, 1000),
                'tol': Real(1e-5, 1e-2, prior='log-uniform')
            }


c:\python_env\brain_age_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Clasificador

In [3]:
from base_classifier import BaseClassifier
from skopt.space import Real, Categorical, Integer
import xgboost as xgb

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold


class XGBoostClassifier(BaseClassifier):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="Lasso"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = xgb.XGBClassifier
        if params is None:
            self.params = {
            'objective': 'binary:logistic',  # Clasificación binaria
            'eval_metric': 'logloss',       # Métrica de evaluación
            'learning_rate': 0.1,           # Tasa de aprendizaje
            'max_depth': 6,                 # Profundidad máxima del árbol
            'subsample': 0.8,               # Proporción de muestras utilizadas por árbol
            'colsample_bytree': 0.8,        # Proporción de características utilizadas por árbol
            'reg_alpha': 0.1,               # Regularización L1
            'reg_lambda': 1.0,              # Regularización L2
            'gamma': 0.0,                   # Mínima reducción de pérdida para crear una división
            'min_child_weight': 1,          # Mínimo peso de la suma de los hijos para una división
            'n_estimators': 100,            # Número de árboles en el modelo
            'scale_pos_weight': 1.0         # Para datos desbalanceados
        }
        if params_space is None:
            self.params_space = {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            }

# Instancia de modelos 

In [4]:
from Plotter import Plotter
from sklearn.preprocessing import MinMaxScaler,StandardScaler

model_reg = ElasticNetRegressor()

Plotters = Plotter()

# Parametros de Plot
colorset = 'darkorange'
nameset = 'ElasticNet'

#parametros de scaler
#1:sin scaler 2:Zscore 3:MinMax
Scaler_reg_train=2

#scaler = MinMaxScaler()
Scaler_reg = StandardScaler()

model_cls = XGBoostClassifier()


In [5]:
#72 features
features = ['Alpha2_canon_EPP_HPC_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_EPP_OCC_left', 'Alpha2_canon_EPP_OCC_right', 'Alpha2_canon_RPD_HPC_left', 'Alpha2_canon_RPD_HPC_right', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_RPD_CING_right', 'Alpha2_canon_RPD_PARIET_left', 'Alpha2_canon_RPD_PARIET_right', 'Alpha2_canon_RPD_OCC_left', 'Alpha2_canon_RPD_OCC_right', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_EPP_OCC_right', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_RPD_HPC_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_RPD_CING_right', 'High_subj_spec_RPD_PARIET_left', 'High_subj_spec_RPD_PARIET_right', 'High_subj_spec_RPD_OCC_left', 'High_subj_spec_RPD_OCC_right', 'Low_subj_spec_EPP_HPC_left', 'Low_subj_spec_EPP_HPC_right', 'Low_subj_spec_EPP_CING_left', 'Low_subj_spec_EPP_CING_right', 'Low_subj_spec_EPP_PARIET_left', 'Low_subj_spec_EPP_PARIET_right', 'Low_subj_spec_EPP_OCC_left', 'Low_subj_spec_EPP_OCC_right', 'Low_subj_spec_RPD_HPC_left', 'Low_subj_spec_RPD_HPC_right', 'Low_subj_spec_RPD_CING_left', 'Low_subj_spec_RPD_CING_right', 'Low_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_right', 'TF_ORB_left', 'TF_ORB_right', 'TF_IFG_left', 'TF_IFG_right', 'TF_MFG_left', 'TF_MFG_right', 'TF_SFG_left', 'TF_SFG_right', 'TF_HPC_left', 'TF_HPC_right', 'TF_INS_left', 'TF_INS_right', 'IAF_ORB_left', 'IAF_ORB_right', 'IAF_IFG_left', 'IAF_IFG_right', 'IAF_MFG_left', 'IAF_MFG_right', 'IAF_SFG_left', 'IAF_SFG_right', 'IAF_HPC_left', 'IAF_HPC_right', 'IAF_INS_left', 'IAF_INS_right', 'IAF_OCC_left', 'IAF_OCC_right']

# Data

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pickle


file_path_CN = f'{path}Brain_Age_Clock/data_freq/CN_freq.xlsx'
file_path_AD = f'{path}Brain_Age_Clock/data_freq/AD_freq.xlsx'
file_path_FTD = f'{path}Brain_Age_Clock/data_freq/FTD_freq.xlsx'
file_path_MCI = f'{path}Brain_Age_Clock/data_freq/MCI_freq.xlsx'

df_CN = pd.read_excel(file_path_CN)
df_AD = pd.read_excel(file_path_AD)
df_FTD = pd.read_excel(file_path_FTD)
df_MCI = pd.read_excel(file_path_MCI)

df_CN_filtrado = df_CN[(df_CN['Age'] >=50) & (df_CN['Age'] <= 90)].reset_index(drop=True)
df_AD_filtrado = df_AD[(df_AD['Age'] >= 50) & (df_AD['Age'] <= 90)].reset_index(drop=True)
df_FTD_filtrado = df_FTD[(df_FTD['Age'] >= 50) & (df_FTD['Age'] <= 90)].reset_index(drop=True)
df_MCI_filtrado = df_MCI[(df_MCI['Age'] >= 50) & (df_MCI['Age'] <= 90)].reset_index(drop=True)


In [7]:
X_AD = df_AD_filtrado[features]
y_AD = df_AD_filtrado["Age"]
ID_AD = df_AD_filtrado["ID_unique"]

X_FTD = df_FTD_filtrado[features]
y_FTD = df_FTD_filtrado["Age"]
ID_FTD = df_FTD_filtrado["ID_unique"]

X_MCI = df_MCI_filtrado[features]
y_MCI = df_MCI_filtrado["Age"]
ID_MCI = df_MCI_filtrado["ID_unique"]

In [8]:
# Definir las regiones para cada subconjunto
regiones_1 = [ "Turquia"]
regiones_2 = ["Turquia","Europa", "America"]

# Filtrar las filas del dataframe por región
df_CN_1 = df_CN_filtrado[df_CN_filtrado["Region"].isin(regiones_1)]
df_CN_2 = df_CN_filtrado[df_CN_filtrado["Region"].isin(regiones_2)]

# Seleccionar las columnas correspondientes para cada conjunto
X_CN_1 = df_CN_1[features]
y_CN_1 = df_CN_1["Age"]
ID_CN_1 = df_CN_1["ID_unique"]


X_CN_2 = df_CN_2[features]
y_CN_2 = df_CN_2["Age"]
ID_CN_2 = df_CN_2["ID_unique"]

scaler = StandardScaler()
scaler.fit(X_CN_1)

X_CN_1_scaled = scaler.transform(X_CN_1)
X_CN_1_scaled = pd.DataFrame(X_CN_1_scaled, columns=X_CN_1.columns)

df_concatenado_CN = pd.concat([X_CN_1, y_CN_1, ID_CN_1], axis=1, ignore_index=False)

In [9]:
best_params = {'alpha': 0.1,
 'l1_ratio': 0.1,
 'max_iter': 1000,
 'tol': 0.01}

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

results = []

# Iterar por cada característica
for feature in features:
    # Crear datasets con solo la característica actual y la columna objetivo
    X_CN_ = X_CN_1[[feature]]
    
    # Dividir en train/test
    #opt_model, best_params = model_reg.search_best_model (X=X_CN_, y=y_CN_1, n_iter_=10, scoring_metric='r2')
    
    

    # Entrenar modelo (ejemplo con regresión lineal)
    #model = ...  # Tu modelo aquí
    results_labels_df_CN_train, results_labels_df_CN_test,  results_model, results_per_fold_CN_train, results_per_fold_CN_test, df_CN_avg_train,df_CN_avg_test= model_reg.trainer(
        X=X_CN_,
        y=y_CN_1,
        ID_label='ID_unique',
        ID=ID_CN_1,
        n_splits=10,
        n_iterations=20,
        params_=best_params
    )

    

    results_labels_df_AD_test, results_per_fold_AD_test, df_AD_avg_test = model_reg.test(X=X_AD[[feature]], y=y_AD, ID= ID_AD, ID_label='ID_unique', n_splits=10, n_iterations=20,result_model=results_model)
    results_labels_df_FTD_test, results_per_fold_FTD_test, df_FTD_avg_test = model_reg.test(X=X_FTD[[feature]], y=y_FTD, ID= ID_FTD, ID_label='ID_unique', n_splits=10, n_iterations=20,result_model=results_model)
    results_labels_df_MCI_test, results_per_fold_MCI_test, df_MCI_avg_test = model_reg.test(X=X_MCI[[feature]], y=y_MCI, ID= ID_MCI, ID_label='ID_unique', n_splits=10, n_iterations=20,result_model=results_model)



    df_CN_1_seleccion = df_CN_filtrado[['ID_unique', 'Country_unique','Region', 'Age', 'Sex', 'Educ', 'MMSE', 'HDI', 'GINI', 'ODQ']]
    df_CN_1_combined = df_CN_avg_test.merge(df_CN_1_seleccion, on='ID_unique', how='left')

    df_AD_seleccion = df_AD_filtrado[['ID_unique', 'Country_unique','Region', 'Age', 'Sex', 'Educ', 'MMSE',  'GINI', 'ODQ']]
    df_AD_combined = df_AD_avg_test.merge(df_AD_seleccion, on='ID_unique', how='left')

    df_FTD_seleccion = df_FTD_filtrado[['ID_unique', 'Country_unique','Region', 'Age', 'Sex', 'Educ', 'MMSE', 'GINI', 'ODQ']]
    df_FTD_combined = df_FTD_avg_test.merge(df_FTD_seleccion, on='ID_unique', how='left')

    df_MCI_seleccion = df_MCI_filtrado[['ID_unique', 'Country_unique', 'Region','Age', 'Sex', 'Educ', 'MMSE',  'GINI', 'ODQ']]
    df_MCI_combined = df_MCI_avg_test.merge(df_MCI_seleccion, on='ID_unique', how='left')

  
    # Calcular métricas en el train set
    r2 = r2_score(df_CN_avg_train['y_labels'], df_CN_avg_train['y_pred'])
    mae = mean_absolute_error(df_CN_avg_train['y_labels'], df_CN_avg_train['y_pred'])
    


    # Calcular diferencias GAP promediada

    avg_gap_CN = np.mean(df_CN_avg_test['GAP_corrected'])

    avg_gap_AD = np.mean(df_AD_avg_test['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_FTD = np.mean(df_FTD_avg_test['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_MCI = np.mean(df_MCI_avg_test['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    
    avg_gap_CN_America = np.mean(np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'America']['GAP_corrected']))
    avg_gap_CN_Europa = np.mean(np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Europa']['GAP_corrected']))
    avg_gap_CN_Turquia = np.mean(np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Turquia']['GAP_corrected']))

    avg_gap_AD_America = np.mean(df_AD_combined[df_AD_combined['Region'] == 'America']['GAP_corrected']) - np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'America']['GAP_corrected'])
    avg_gap_AD_Europa = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Europa']['GAP_corrected']) - np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Europa']['GAP_corrected'])
    avg_gap_AD_Turquia = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Turquia']['GAP_corrected']) - np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Turquia']['GAP_corrected'])

    avg_gap_AD_America_rel = np.mean(df_AD_combined[df_AD_combined['Region'] == 'America']['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_AD_Europa_rel = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Europa']['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_AD_Turquia_rel = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Turquia']['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])


    # Guardar resultados
    results.append({
        "Feature": feature,
        "R2": r2,
        "MAE": mae,
        "Avg_GAP_CN": avg_gap_CN,
        "Avg_GAP_AD": avg_gap_AD,
        "Avg_GAP_FTD": avg_gap_FTD,
        "Avg_GAP_MCI": avg_gap_MCI,
        "Avg_GAP_CN_America": avg_gap_CN_America,
        "Avg_GAP_CN_Europa": avg_gap_CN_Europa,
        "Avg_GAP_CN_Turquia": avg_gap_CN_Turquia,
        "Avg_GAP_AD_America": avg_gap_AD_America,
        "Avg_GAP_AD_Europa": avg_gap_AD_Europa,
        "Avg_GAP_AD_Turquia": avg_gap_AD_Turquia,
        "Avg_GAP_AD_America_Rel": avg_gap_AD_America_rel,
        "Avg_GAP_AD_Europa_Rel": avg_gap_AD_Europa_rel,
        "Avg_GAP_AD_Turquia_Rel": avg_gap_AD_Turquia_rel,
    })


# Convertir resultados a dataframe
results_df = pd.DataFrame(results)

# Ordenar por la métrica que te interese, por ejemplo, menor MAE
results_df = results_df.sort_values(by="MAE")

C:\Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/Base_Models\base_regressor.py:207: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_labels_df_train = pd.concat([results_labels_df_train, temp_df_train], ignore_index=True)
C:\Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/Base_Models\base_regressor.py:209: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_labels_df_test = pd.concat([resul

In [11]:
results_df.to_excel("results_metrics_all_sin_cuba_Solo_Turquia_train.xlsx", index=False)

In [12]:
results_df

,Feature,R2,MAE,Avg_GAP_CN,Avg_GAP_AD,Avg_GAP_FTD,Avg_GAP_MCI,Avg_GAP_CN_America,Avg_GAP_CN_Europa,Avg_GAP_CN_Turquia,Avg_GAP_AD_America,Avg_GAP_AD_Europa,Avg_GAP_AD_Turquia,Avg_GAP_AD_America_Rel,Avg_GAP_AD_Europa_Rel,Avg_GAP_AD_Turquia_Rel
14,High_subj_spec_EPP_HPC_left,0.060945,6.507462,0.002931,0.749124,0.840673,0.963468,NaN,NaN,0.002931,NaN,NaN,-0.530810,1.339862,NaN,-0.530810
22,High_subj_spec_RPD_HPC_left,0.060945,6.507462,0.002931,0.749124,0.840673,0.963468,NaN,NaN,0.002931,NaN,NaN,-0.530810,1.339862,NaN,-0.530810
0,Alpha2_canon_EPP_HPC_left,0.048109,6.525537,0.002156,0.390544,0.457317,0.827440,NaN,NaN,0.002156,NaN,NaN,-0.558728,0.828669,NaN,-0.558728
6,Alpha2_canon_RPD_HPC_left,0.048109,6.525537,0.002156,0.390544,0.457317,0.827440,NaN,NaN,0.002156,NaN,NaN,-0.558728,0.828669,NaN,-0.558728
38,Low_subj_spec_RPD_HPC_left,0.047020,6.547687,0.002191,0.951377,1.058195,0.998529,NaN,NaN,0.002191,NaN,NaN,-0.436862,1.592102,NaN,-0.436862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,High_subj_spec_RPD_CING_right,0.005264,6.724418,0.006940,0.186177,0.089394,0.200034,NaN,NaN,0.006940,NaN,NaN,0.026626,0.259817,NaN,0.026626
60,IAF_IFG_left,0.005908,6.724724,-0.004808,0.151429,0.282070,0.220447,NaN,NaN,-0.004808,NaN,NaN,-0.101796,0.268303,NaN,-0.101796
41,Low_subj_spec_RPD_CING_right,0.005693,6.725137,0.006383,0.213430,0.111582,0.221622,NaN,NaN,0.006383,NaN,NaN,0.033026,0.296694,NaN,0.033026
68,IAF_INS_left,0.005543,6.726303,-0.005915,0.161190,0.219598,0.173552,NaN,NaN,-0.005915,NaN,NaN,-0.041820,0.254887,NaN,-0.041820


In [13]:
filtered_features_Turquia = results_df.loc[ (results_df['Avg_GAP_AD_Turquia'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_Turquia = filtered_features_Turquia.tolist()

print(filtered_features_list_Turquia)

['TF_HPC_right', 'TF_IFG_right', 'TF_ORB_right', 'TF_MFG_left', 'TF_ORB_left', 'TF_IFG_left', 'TF_INS_left', 'TF_HPC_left', 'IAF_OCC_left', 'Alpha2_canon_RPD_CING_left', 'TF_INS_right', 'TF_SFG_left', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'TF_MFG_right', 'IAF_OCC_right', 'Alpha2_canon_RPD_CING_right', 'Low_subj_spec_EPP_OCC_right', 'Low_subj_spec_EPP_PARIET_right', 'IAF_SFG_left', 'Low_subj_spec_EPP_OCC_left', 'Low_subj_spec_RPD_CING_left', 'IAF_MFG_left', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_RPD_CING_right', 'Low_subj_spec_RPD_CING_right']


In [14]:
filtered_features_Turquia_Rel = results_df.loc[ (results_df['Avg_GAP_AD_Turquia_Rel'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_Turquia_Rel = filtered_features_Turquia_Rel.tolist()

print(filtered_features_list_Turquia_Rel)

['TF_HPC_right', 'TF_IFG_right', 'TF_ORB_right', 'TF_MFG_left', 'TF_ORB_left', 'TF_IFG_left', 'TF_INS_left', 'TF_HPC_left', 'IAF_OCC_left', 'Alpha2_canon_RPD_CING_left', 'TF_INS_right', 'TF_SFG_left', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'TF_MFG_right', 'IAF_OCC_right', 'Alpha2_canon_RPD_CING_right', 'Low_subj_spec_EPP_OCC_right', 'Low_subj_spec_EPP_PARIET_right', 'IAF_SFG_left', 'Low_subj_spec_EPP_OCC_left', 'Low_subj_spec_RPD_CING_left', 'IAF_MFG_left', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_RPD_CING_right', 'Low_subj_spec_RPD_CING_right']
